In [1]:
x=1

In [2]:
import random
import scanpy as sc
import numpy as np
from sklearn.model_selection import KFold

In [3]:
adata = sc.read('/lustre/groups/ml01/projects/2022_multigrate_anastasia.litinetskaya/snakemake/hyperparam/data/raw/HLCA_full_v1.1_to_share.h5ad', backed='r')
adata

AnnData object with n_obs × n_vars = 2282447 × 59586 backed at '/lustre/groups/ml01/projects/2022_multigrate_anastasia.litinetskaya/snakemake/hyperparam/data/raw/HLCA_full_v1.1_to_share.h5ad'
    obs: "3'_or_5'", 'BMI', 'age_or_mean_of_age_range', 'age_range', 'anatomical_region_ccf_score', 'ancestry', 'ann_coarse_for_GWAS_and_modeling', 'ann_finest_level', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'assay', 'cause_of_death', 'cell_type', 'core_or_extension', 'dataset', 'development_stage', 'disease', 'donor_id', 'fresh_or_frozen', 'log10_total_counts', 'lung_condition', 'mixed_ancestry', 'original_ann_level_1', 'original_ann_level_2', 'original_ann_level_3', 'original_ann_level_4', 'original_ann_level_5', 'original_ann_nonharmonized', 'reannotation_type', 'sample', 'scanvi_label', 'sequencing_platform', 'sex', 'smoking_status', 'study', 'subject_type', 'suspension_type', 'tissue', 'tissue_coarse_unharmonized', 'tissue_detailed_unharmonized', 'tissue_dis

In [4]:
tmp = sc.AnnData(adata.layers['counts'])
tmp.obs = adata.obs
tmp.var_names = adata.var_names
adata = tmp
del tmp
adata

AnnData object with n_obs × n_vars = 2282447 × 59586
    obs: "3'_or_5'", 'BMI', 'age_or_mean_of_age_range', 'age_range', 'anatomical_region_ccf_score', 'ancestry', 'ann_coarse_for_GWAS_and_modeling', 'ann_finest_level', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'assay', 'cause_of_death', 'cell_type', 'core_or_extension', 'dataset', 'development_stage', 'disease', 'donor_id', 'fresh_or_frozen', 'log10_total_counts', 'lung_condition', 'mixed_ancestry', 'original_ann_level_1', 'original_ann_level_2', 'original_ann_level_3', 'original_ann_level_4', 'original_ann_level_5', 'original_ann_nonharmonized', 'reannotation_type', 'sample', 'scanvi_label', 'sequencing_platform', 'sex', 'smoking_status', 'study', 'subject_type', 'suspension_type', 'tissue', 'tissue_coarse_unharmonized', 'tissue_detailed_unharmonized', 'tissue_dissociation_protocol', 'tissue_level_2', 'tissue_level_3', 'tissue_sampling_method', 'total_counts', 'transf_ann_level_1_label', 'transf_ann_

# get rid of celltypes

In [5]:
adata = adata[~adata.obs['transf_ann_level_3_label'].isin(['2_Hematopoietic stem cells', '2_Mesothelium', '2_Smooth muscle', 'Rare'])].copy()
adata

AnnData object with n_obs × n_vars = 2250578 × 59586
    obs: "3'_or_5'", 'BMI', 'age_or_mean_of_age_range', 'age_range', 'anatomical_region_ccf_score', 'ancestry', 'ann_coarse_for_GWAS_and_modeling', 'ann_finest_level', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'assay', 'cause_of_death', 'cell_type', 'core_or_extension', 'dataset', 'development_stage', 'disease', 'donor_id', 'fresh_or_frozen', 'log10_total_counts', 'lung_condition', 'mixed_ancestry', 'original_ann_level_1', 'original_ann_level_2', 'original_ann_level_3', 'original_ann_level_4', 'original_ann_level_5', 'original_ann_nonharmonized', 'reannotation_type', 'sample', 'scanvi_label', 'sequencing_platform', 'sex', 'smoking_status', 'study', 'subject_type', 'suspension_type', 'tissue', 'tissue_coarse_unharmonized', 'tissue_detailed_unharmonized', 'tissue_dissociation_protocol', 'tissue_level_2', 'tissue_level_3', 'tissue_sampling_method', 'total_counts', 'transf_ann_level_1_label', 'transf_ann_

# clean conditions

In [6]:
condition_simplifier = {
        "Healthy (tumor adjacent)": "Healthy",
        "Large Cell Carcinoma": "Carcinoma",
        "Lung adenocarcinoma": "Carcinoma",
        "Pleiomorphic Carcinoma": "Carcinoma",
        "Squamous Cell Carcinoma": "Carcinoma",
        "HP": "ILD",
        "Myositis-associated ILD": "ILD",
        "NSIP": "ILD",
        "Systemic sclerosis-associated ILD" : "ILD",
    }

In [7]:
adata.obs['lung_condition_coarse'] = adata.obs['lung_condition'].replace(condition_simplifier)
adata.obs['lung_condition_coarse'].cat.categories

Index(['CF', 'COPD', 'COVID-19', 'COVID-19 moderate', 'COVID-19 severe',
       'Chronic rhinitis', 'End-stage lung fibrosis, unknown etiology',
       'Healthy', 'ILD', 'IPF', 'Carcinoma', 'Lymphangioleiomyomatosis',
       'Pneumonia non-COVID', 'Sarcoidosis',
       'anti-CD3 anti-CD28 antibody stimulated'],
      dtype='object')

In [8]:
adata.obs[['lung_condition_coarse', 'sample']].drop_duplicates()['lung_condition_coarse'].value_counts()

Healthy                                      429
COVID-19                                      73
IPF                                           67
Carcinoma                                     44
COPD                                          24
CF                                            22
ILD                                           22
Chronic rhinitis                              18
Pneumonia non-COVID                           13
COVID-19 severe                                6
End-stage lung fibrosis, unknown etiology      4
COVID-19 moderate                              3
Lymphangioleiomyomatosis                       2
Sarcoidosis                                    2
anti-CD3 anti-CD28 antibody stimulated         2
Name: lung_condition_coarse, dtype: int64

In [9]:
conditions_to_exclude = [
    "End-stage lung fibrosis, unknown etiology",
    "Lymphangioleiomyomatosis",
    "Sarcoidosis",
    "anti-CD3 anti-CD28 antibody stimulated",
]

In [10]:
adata = adata[~adata.obs['lung_condition_coarse'].isin(conditions_to_exclude)].copy()
adata

AnnData object with n_obs × n_vars = 2212137 × 59586
    obs: "3'_or_5'", 'BMI', 'age_or_mean_of_age_range', 'age_range', 'anatomical_region_ccf_score', 'ancestry', 'ann_coarse_for_GWAS_and_modeling', 'ann_finest_level', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'assay', 'cause_of_death', 'cell_type', 'core_or_extension', 'dataset', 'development_stage', 'disease', 'donor_id', 'fresh_or_frozen', 'log10_total_counts', 'lung_condition', 'mixed_ancestry', 'original_ann_level_1', 'original_ann_level_2', 'original_ann_level_3', 'original_ann_level_4', 'original_ann_level_5', 'original_ann_nonharmonized', 'reannotation_type', 'sample', 'scanvi_label', 'sequencing_platform', 'sex', 'smoking_status', 'study', 'subject_type', 'suspension_type', 'tissue', 'tissue_coarse_unharmonized', 'tissue_detailed_unharmonized', 'tissue_dissociation_protocol', 'tissue_level_2', 'tissue_level_3', 'tissue_sampling_method', 'total_counts', 'transf_ann_level_1_label', 'transf_ann_

# check what is our query

In [11]:
adata[adata.obs['lung_condition'].isin(['COVID-19 moderate', 'COVID-19 severe'])].obs['study']

AAGGTTCGTGCTAGCC_8_liao     Zhang_2021
GGACAAGTCGTGACAT_9_liao     Zhang_2021
CCATTCGAGCTCCTCT_7_liao     Zhang_2021
GGTGCGTGTGTAATGA_8_liao     Zhang_2021
GCGCAACGTGGACGAT_7_liao     Zhang_2021
                               ...    
TTCTCCTTCCTTCAAT_13_liao    Zhang_2021
GGGCATCCATACCATG_9_liao     Zhang_2021
CGGAGCTTCCAGAGGA_8_liao     Zhang_2021
CCCAGTTGTTACGGAG_13_liao    Zhang_2021
GTTCTCGTCTGGCGAC_8_liao     Zhang_2021
Name: study, Length: 43316, dtype: category
Categories (1, object): ['Zhang_2021']

In [12]:
adata[adata.obs['dataset'] == 'Zhang_2021'].obs['lung_condition']

AAGGTTCGTGCTAGCC_8_liao       COVID-19 severe
GGACAAGTCGTGACAT_9_liao       COVID-19 severe
CCATTCGAGCTCCTCT_7_liao     COVID-19 moderate
GGTGCGTGTGTAATGA_8_liao       COVID-19 severe
GGGTCTGTCTAACTGG_2_liao               Healthy
                                  ...        
GGGCATCCATACCATG_9_liao       COVID-19 severe
CGGAGCTTCCAGAGGA_8_liao       COVID-19 severe
CCCAGTTGTTACGGAG_13_liao      COVID-19 severe
GAAGCAGGTTAAAGTG_2_liao               Healthy
GTTCTCGTCTGGCGAC_8_liao       COVID-19 severe
Name: lung_condition, Length: 62453, dtype: category
Categories (3, object): ['COVID-19 moderate', 'COVID-19 severe', 'Healthy']

In [22]:
adata.write('/lustre/groups/ml01/projects/2022_multigrate_anastasia.litinetskaya/snakemake/hyperparam/data/pp/hlca.h5ad')

# pearson hvg

In [13]:
sc.experimental.pp.highly_variable_genes(
    adata, flavor="pearson_residuals", n_top_genes=2000, batch_key='dataset',
)

In [14]:
adata_pearson = adata[:, adata.var['highly_variable']].copy()
adata_pearson

AnnData object with n_obs × n_vars = 2212137 × 2000
    obs: "3'_or_5'", 'BMI', 'age_or_mean_of_age_range', 'age_range', 'anatomical_region_ccf_score', 'ancestry', 'ann_coarse_for_GWAS_and_modeling', 'ann_finest_level', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'assay', 'cause_of_death', 'cell_type', 'core_or_extension', 'dataset', 'development_stage', 'disease', 'donor_id', 'fresh_or_frozen', 'log10_total_counts', 'lung_condition', 'mixed_ancestry', 'original_ann_level_1', 'original_ann_level_2', 'original_ann_level_3', 'original_ann_level_4', 'original_ann_level_5', 'original_ann_nonharmonized', 'reannotation_type', 'sample', 'scanvi_label', 'sequencing_platform', 'sex', 'smoking_status', 'study', 'subject_type', 'suspension_type', 'tissue', 'tissue_coarse_unharmonized', 'tissue_detailed_unharmonized', 'tissue_dissociation_protocol', 'tissue_level_2', 'tissue_level_3', 'tissue_sampling_method', 'total_counts', 'transf_ann_level_1_label', 'transf_ann_l

## separate query

In [16]:
adata_pearson_query = adata_pearson[adata_pearson.obs['dataset'] == 'Zhang_2021'].copy()
adata_pearson = adata_pearson[adata_pearson.obs['dataset'] != 'Zhang_2021'].copy()

## create splits

In [17]:
samples = adata_pearson.obs['sample'].cat.categories.values
len(samples)

709

In [18]:
kf = KFold(n_splits=5, random_state=1, shuffle=True)

In [19]:
train_test_sizes = []

for i, (train_index, test_index) in enumerate(kf.split(samples)):
    train_samples = samples[train_index]
    test_samples = samples[test_index]
    adata_pearson.obs.loc[adata_pearson.obs['sample'].isin(train_samples), f'split{i}'] = 'train'
    adata_pearson.obs.loc[adata_pearson.obs['sample'].isin(test_samples), f'split{i}'] = 'val'
    adata_train = adata_pearson[adata_pearson.obs[f'split{i}'] == 'train'].copy()
    adata_test = adata_pearson[adata_pearson.obs[f'split{i}'] == 'val'].copy()
    train_conditions = set(adata_train.obs['lung_condition_coarse'].cat.categories)
    test_conditions = set(adata_test.obs['lung_condition_coarse'].cat.categories)
    print(train_conditions)
    print(test_conditions)
    assert test_conditions.issubset(train_conditions)
    train_test_sizes.append((len(adata_train), len(adata_test)))

{'IPF', 'COPD', 'Carcinoma', 'CF', 'Chronic rhinitis', 'Healthy', 'COVID-19', 'ILD', 'Pneumonia non-COVID'}
{'IPF', 'COPD', 'Carcinoma', 'CF', 'Chronic rhinitis', 'Healthy', 'COVID-19', 'ILD', 'Pneumonia non-COVID'}
{'IPF', 'COPD', 'Carcinoma', 'CF', 'Chronic rhinitis', 'Healthy', 'COVID-19', 'ILD', 'Pneumonia non-COVID'}
{'IPF', 'COPD', 'Carcinoma', 'CF', 'Chronic rhinitis', 'Healthy', 'COVID-19', 'ILD', 'Pneumonia non-COVID'}
{'IPF', 'COPD', 'Carcinoma', 'CF', 'Chronic rhinitis', 'Healthy', 'COVID-19', 'ILD', 'Pneumonia non-COVID'}
{'IPF', 'COPD', 'Carcinoma', 'CF', 'Chronic rhinitis', 'Healthy', 'COVID-19', 'ILD', 'Pneumonia non-COVID'}
{'IPF', 'COPD', 'Carcinoma', 'CF', 'Chronic rhinitis', 'Healthy', 'COVID-19', 'ILD', 'Pneumonia non-COVID'}
{'IPF', 'COPD', 'Carcinoma', 'CF', 'Chronic rhinitis', 'Healthy', 'COVID-19', 'ILD', 'Pneumonia non-COVID'}
{'IPF', 'COPD', 'Carcinoma', 'CF', 'Chronic rhinitis', 'Healthy', 'COVID-19', 'ILD', 'Pneumonia non-COVID'}
{'IPF', 'COPD', 'Carcinoma',

In [20]:
print(train_test_sizes)

[(1733726, 415958), (1751019, 398665), (1666343, 483341), (1730932, 418752), (1716716, 432968)]


In [21]:
adata_pearson.write('/lustre/groups/ml01/projects/2022_multigrate_anastasia.litinetskaya/snakemake/hyperparam/data/pp/hlca_hvg_pearson.h5ad')
adata_pearson_query.write('/lustre/groups/ml01/projects/2022_multigrate_anastasia.litinetskaya/snakemake/hyperparam/data/pp/hlca_hvg_pearson_query.h5ad')

# hvg

In [3]:
adata = sc.read('/lustre/groups/ml01/projects/2022_multigrate_anastasia.litinetskaya/snakemake/hyperparam/data/pp/hlca.h5ad')

In [4]:
adata.layers['counts'] = adata.X.copy()

In [5]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [6]:
sc.pp.highly_variable_genes(
    adata, n_top_genes=2000, batch_key='dataset'
)

/lustre/groups/ml01/workspace/anastasia.litinetskaya/miniconda3/envs/multimil/lib/python3.10/site-packages/scanpy/preprocessing/_highly_variable_genes.py:478: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hvg = hvg.append(missing_hvg, ignore_index=True)
/lustre/groups/ml01/workspace/anastasia.litinetskaya/miniconda3/envs/multimil/lib/python3.10/site-packages/scanpy/preprocessing/_highly_variable_genes.py:478: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hvg = hvg.append(missing_hvg, ignore_index=True)
/lustre/groups/ml01/workspace/anastasia.litinetskaya/miniconda3/envs/multimil/lib/python3.10/site-packages/scanpy/preprocessing/_highly_variable_genes.py:478: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hvg = hvg.append(mis

/lustre/groups/ml01/workspace/anastasia.litinetskaya/miniconda3/envs/multimil/lib/python3.10/site-packages/scanpy/preprocessing/_highly_variable_genes.py:478: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hvg = hvg.append(missing_hvg, ignore_index=True)
/lustre/groups/ml01/workspace/anastasia.litinetskaya/miniconda3/envs/multimil/lib/python3.10/site-packages/scanpy/preprocessing/_highly_variable_genes.py:478: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hvg = hvg.append(missing_hvg, ignore_index=True)
/lustre/groups/ml01/workspace/anastasia.litinetskaya/miniconda3/envs/multimil/lib/python3.10/site-packages/scanpy/preprocessing/_highly_variable_genes.py:478: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hvg = hvg.append(mis

In [7]:
adata_hvg = adata[:, adata.var['highly_variable']].copy()
adata_hvg

AnnData object with n_obs × n_vars = 2212137 × 2000
    obs: "3'_or_5'", 'BMI', 'age_or_mean_of_age_range', 'age_range', 'anatomical_region_ccf_score', 'ancestry', 'ann_coarse_for_GWAS_and_modeling', 'ann_finest_level', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'assay', 'cause_of_death', 'cell_type', 'core_or_extension', 'dataset', 'development_stage', 'disease', 'donor_id', 'fresh_or_frozen', 'log10_total_counts', 'lung_condition', 'mixed_ancestry', 'original_ann_level_1', 'original_ann_level_2', 'original_ann_level_3', 'original_ann_level_4', 'original_ann_level_5', 'original_ann_nonharmonized', 'reannotation_type', 'sample', 'scanvi_label', 'sequencing_platform', 'sex', 'smoking_status', 'study', 'subject_type', 'suspension_type', 'tissue', 'tissue_coarse_unharmonized', 'tissue_detailed_unharmonized', 'tissue_dissociation_protocol', 'tissue_level_2', 'tissue_level_3', 'tissue_sampling_method', 'total_counts', 'transf_ann_level_1_label', 'transf_ann_l

In [8]:
adata_hvg.X = adata_hvg.layers['counts'].copy()
del adata_hvg.layers

## separate query

In [9]:
adata_hvg_query = adata_hvg[adata_hvg.obs['dataset'] == 'Zhang_2021'].copy()
adata_hvg = adata_hvg[adata_hvg.obs['dataset'] != 'Zhang_2021'].copy()

## add same splits

In [10]:
adata_pearson = sc.read('/lustre/groups/ml01/projects/2022_multigrate_anastasia.litinetskaya/snakemake/hyperparam/data/pp/hlca_hvg_pearson.h5ad')
adata_pearson

AnnData object with n_obs × n_vars = 2149684 × 2000
    obs: "3'_or_5'", 'BMI', 'age_or_mean_of_age_range', 'age_range', 'anatomical_region_ccf_score', 'ancestry', 'ann_coarse_for_GWAS_and_modeling', 'ann_finest_level', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'assay', 'cause_of_death', 'cell_type', 'core_or_extension', 'dataset', 'development_stage', 'disease', 'donor_id', 'fresh_or_frozen', 'log10_total_counts', 'lung_condition', 'mixed_ancestry', 'original_ann_level_1', 'original_ann_level_2', 'original_ann_level_3', 'original_ann_level_4', 'original_ann_level_5', 'original_ann_nonharmonized', 'reannotation_type', 'sample', 'scanvi_label', 'sequencing_platform', 'sex', 'smoking_status', 'study', 'subject_type', 'suspension_type', 'tissue', 'tissue_coarse_unharmonized', 'tissue_detailed_unharmonized', 'tissue_dissociation_protocol', 'tissue_level_2', 'tissue_level_3', 'tissue_sampling_method', 'total_counts', 'transf_ann_level_1_label', 'transf_ann_l

In [11]:
adata_hvg.obs = adata_hvg.obs.join(adata_pearson.obs[[f'split{i}' for i in range(5)]])

In [12]:
adata_hvg.write('/lustre/groups/ml01/projects/2022_multigrate_anastasia.litinetskaya/snakemake/hyperparam/data/pp/hlca_hvg.h5ad')
adata_hvg_query.write('/lustre/groups/ml01/projects/2022_multigrate_anastasia.litinetskaya/snakemake/hyperparam/data/pp/hlca_hvg_query.h5ad')

# downsample Healthy

## pearson hvg

In [13]:
adata = sc.read('/lustre/groups/ml01/projects/2022_multigrate_anastasia.litinetskaya/snakemake/hyperparam/data/pp/hlca_hvg_pearson.h5ad')
adata

AnnData object with n_obs × n_vars = 2149684 × 2000
    obs: "3'_or_5'", 'BMI', 'age_or_mean_of_age_range', 'age_range', 'anatomical_region_ccf_score', 'ancestry', 'ann_coarse_for_GWAS_and_modeling', 'ann_finest_level', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'assay', 'cause_of_death', 'cell_type', 'core_or_extension', 'dataset', 'development_stage', 'disease', 'donor_id', 'fresh_or_frozen', 'log10_total_counts', 'lung_condition', 'mixed_ancestry', 'original_ann_level_1', 'original_ann_level_2', 'original_ann_level_3', 'original_ann_level_4', 'original_ann_level_5', 'original_ann_nonharmonized', 'reannotation_type', 'sample', 'scanvi_label', 'sequencing_platform', 'sex', 'smoking_status', 'study', 'subject_type', 'suspension_type', 'tissue', 'tissue_coarse_unharmonized', 'tissue_detailed_unharmonized', 'tissue_dissociation_protocol', 'tissue_level_2', 'tissue_level_3', 'tissue_sampling_method', 'total_counts', 'transf_ann_level_1_label', 'transf_ann_l

In [14]:
condition = 'lung_condition_coarse'
sample = 'sample'

In [15]:
adata.obs[[condition, sample]].drop_duplicates().groupby(condition).count()

,sample
lung_condition_coarse,
CF,22
COPD,24
COVID-19,73
Chronic rhinitis,18
Healthy,426
ILD,22
IPF,67
Carcinoma,44
Pneumonia non-COVID,13


In [16]:
healthy_samples = list(adata[adata.obs[condition] == 'Healthy'].obs[sample].cat.categories)
len(healthy_samples)

426

In [17]:
subsamples = random.sample(healthy_samples, 85)

In [18]:
adata = adata[(adata.obs[condition] != 'Healthy') | (adata.obs[sample].isin(subsamples))].copy()
adata

AnnData object with n_obs × n_vars = 1107292 × 2000
    obs: "3'_or_5'", 'BMI', 'age_or_mean_of_age_range', 'age_range', 'anatomical_region_ccf_score', 'ancestry', 'ann_coarse_for_GWAS_and_modeling', 'ann_finest_level', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'assay', 'cause_of_death', 'cell_type', 'core_or_extension', 'dataset', 'development_stage', 'disease', 'donor_id', 'fresh_or_frozen', 'log10_total_counts', 'lung_condition', 'mixed_ancestry', 'original_ann_level_1', 'original_ann_level_2', 'original_ann_level_3', 'original_ann_level_4', 'original_ann_level_5', 'original_ann_nonharmonized', 'reannotation_type', 'sample', 'scanvi_label', 'sequencing_platform', 'sex', 'smoking_status', 'study', 'subject_type', 'suspension_type', 'tissue', 'tissue_coarse_unharmonized', 'tissue_detailed_unharmonized', 'tissue_dissociation_protocol', 'tissue_level_2', 'tissue_level_3', 'tissue_sampling_method', 'total_counts', 'transf_ann_level_1_label', 'transf_ann_l

In [19]:
adata.obs[[condition, sample]].drop_duplicates().groupby(condition).count()

,sample
lung_condition_coarse,
CF,22
COPD,24
COVID-19,73
Chronic rhinitis,18
Healthy,85
ILD,22
IPF,67
Carcinoma,44
Pneumonia non-COVID,13


In [20]:
adata.write('/lustre/groups/ml01/projects/2022_multigrate_anastasia.litinetskaya/snakemake/hyperparam/data/pp/hlca_hvg_pearson_subsampled.h5ad')

## not pearson hvg with same samples

In [13]:
adata_pearson = sc.read('/lustre/groups/ml01/projects/2022_multigrate_anastasia.litinetskaya/snakemake/hyperparam/data/pp/hlca_hvg_pearson_subsampled.h5ad')
adata_pearson

AnnData object with n_obs × n_vars = 1107292 × 2000
    obs: "3'_or_5'", 'BMI', 'age_or_mean_of_age_range', 'age_range', 'anatomical_region_ccf_score', 'ancestry', 'ann_coarse_for_GWAS_and_modeling', 'ann_finest_level', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'assay', 'cause_of_death', 'cell_type', 'core_or_extension', 'dataset', 'development_stage', 'disease', 'donor_id', 'fresh_or_frozen', 'log10_total_counts', 'lung_condition', 'mixed_ancestry', 'original_ann_level_1', 'original_ann_level_2', 'original_ann_level_3', 'original_ann_level_4', 'original_ann_level_5', 'original_ann_nonharmonized', 'reannotation_type', 'sample', 'scanvi_label', 'sequencing_platform', 'sex', 'smoking_status', 'study', 'subject_type', 'suspension_type', 'tissue', 'tissue_coarse_unharmonized', 'tissue_detailed_unharmonized', 'tissue_dissociation_protocol', 'tissue_level_2', 'tissue_level_3', 'tissue_sampling_method', 'total_counts', 'transf_ann_level_1_label', 'transf_ann_l

In [14]:
adata = sc.read('/lustre/groups/ml01/projects/2022_multigrate_anastasia.litinetskaya/snakemake/hyperparam/data/pp/hlca_hvg.h5ad')
adata

AnnData object with n_obs × n_vars = 2149684 × 2000
    obs: "3'_or_5'", 'BMI', 'age_or_mean_of_age_range', 'age_range', 'anatomical_region_ccf_score', 'ancestry', 'ann_coarse_for_GWAS_and_modeling', 'ann_finest_level', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'assay', 'cause_of_death', 'cell_type', 'core_or_extension', 'dataset', 'development_stage', 'disease', 'donor_id', 'fresh_or_frozen', 'log10_total_counts', 'lung_condition', 'mixed_ancestry', 'original_ann_level_1', 'original_ann_level_2', 'original_ann_level_3', 'original_ann_level_4', 'original_ann_level_5', 'original_ann_nonharmonized', 'reannotation_type', 'sample', 'scanvi_label', 'sequencing_platform', 'sex', 'smoking_status', 'study', 'subject_type', 'suspension_type', 'tissue', 'tissue_coarse_unharmonized', 'tissue_detailed_unharmonized', 'tissue_dissociation_protocol', 'tissue_level_2', 'tissue_level_3', 'tissue_sampling_method', 'total_counts', 'transf_ann_level_1_label', 'transf_ann_l

In [15]:
adata = adata[adata_pearson.obs_names].copy()
adata

AnnData object with n_obs × n_vars = 1107292 × 2000
    obs: "3'_or_5'", 'BMI', 'age_or_mean_of_age_range', 'age_range', 'anatomical_region_ccf_score', 'ancestry', 'ann_coarse_for_GWAS_and_modeling', 'ann_finest_level', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'assay', 'cause_of_death', 'cell_type', 'core_or_extension', 'dataset', 'development_stage', 'disease', 'donor_id', 'fresh_or_frozen', 'log10_total_counts', 'lung_condition', 'mixed_ancestry', 'original_ann_level_1', 'original_ann_level_2', 'original_ann_level_3', 'original_ann_level_4', 'original_ann_level_5', 'original_ann_nonharmonized', 'reannotation_type', 'sample', 'scanvi_label', 'sequencing_platform', 'sex', 'smoking_status', 'study', 'subject_type', 'suspension_type', 'tissue', 'tissue_coarse_unharmonized', 'tissue_detailed_unharmonized', 'tissue_dissociation_protocol', 'tissue_level_2', 'tissue_level_3', 'tissue_sampling_method', 'total_counts', 'transf_ann_level_1_label', 'transf_ann_l

In [17]:
condition = 'lung_condition_coarse'
sample = 'sample'

In [18]:
adata.obs[[condition, sample]].drop_duplicates().groupby(condition).count()

,sample
lung_condition_coarse,
CF,22
COPD,24
COVID-19,73
Chronic rhinitis,18
Healthy,85
ILD,22
IPF,67
Carcinoma,44
Pneumonia non-COVID,13


In [19]:
adata.write('/lustre/groups/ml01/projects/2022_multigrate_anastasia.litinetskaya/snakemake/hyperparam/data/pp/hlca_hvg_subsampled.h5ad')

# check how many samples per condition in each train/val split for downsampled

In [6]:
condition = 'lung_condition_coarse'
sample = 'sample'

In [3]:
adata = sc.read('/lustre/groups/ml01/projects/2022_multigrate_anastasia.litinetskaya/snakemake/hyperparam/data/pp/hlca_hvg_pearson_subsampled.h5ad')
adata

AnnData object with n_obs × n_vars = 1107292 × 2000
    obs: "3'_or_5'", 'BMI', 'age_or_mean_of_age_range', 'age_range', 'anatomical_region_ccf_score', 'ancestry', 'ann_coarse_for_GWAS_and_modeling', 'ann_finest_level', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'assay', 'cause_of_death', 'cell_type', 'core_or_extension', 'dataset', 'development_stage', 'disease', 'donor_id', 'fresh_or_frozen', 'log10_total_counts', 'lung_condition', 'mixed_ancestry', 'original_ann_level_1', 'original_ann_level_2', 'original_ann_level_3', 'original_ann_level_4', 'original_ann_level_5', 'original_ann_nonharmonized', 'reannotation_type', 'sample', 'scanvi_label', 'sequencing_platform', 'sex', 'smoking_status', 'study', 'subject_type', 'suspension_type', 'tissue', 'tissue_coarse_unharmonized', 'tissue_detailed_unharmonized', 'tissue_dissociation_protocol', 'tissue_level_2', 'tissue_level_3', 'tissue_sampling_method', 'total_counts', 'transf_ann_level_1_label', 'transf_ann_l

In [8]:
for i in range(5):
    train = adata[adata.obs[f'split{i}'] == 'train'].obs[[condition, sample]].drop_duplicates().groupby(condition).count()
    val = adata[adata.obs[f'split{i}'] == 'val'].obs[[condition, sample]].drop_duplicates().groupby(condition).count()
    print('train')
    print(train)
    print('val')
    print(val)
    print('=========')

train
                       sample
lung_condition_coarse        
CF                         19
COPD                       16
COVID-19                   60
Chronic rhinitis           14
Healthy                    67
ILD                        19
IPF                        48
Carcinoma                  38
Pneumonia non-COVID        11
val
                       sample
lung_condition_coarse        
CF                          3
COPD                        8
COVID-19                   13
Chronic rhinitis            4
Healthy                    18
ILD                         3
IPF                        19
Carcinoma                   6
Pneumonia non-COVID         2
train
                       sample
lung_condition_coarse        
CF                         18
COPD                       20
COVID-19                   59
Chronic rhinitis           15
Healthy                    58
ILD                        15
IPF                        55
Carcinoma                  39
Pneumonia non-COVID     

# save object with scanVI embeds

In [4]:
adata

AnnData object with n_obs × n_vars = 2282447 × 59586 backed at '/lustre/groups/ml01/projects/2022_multigrate_anastasia.litinetskaya/snakemake/hyperparam/data/raw/HLCA_full_v1.1_to_share.h5ad'
    obs: "3'_or_5'", 'BMI', 'age_or_mean_of_age_range', 'age_range', 'anatomical_region_ccf_score', 'ancestry', 'ann_coarse_for_GWAS_and_modeling', 'ann_finest_level', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'assay', 'cause_of_death', 'cell_type', 'core_or_extension', 'dataset', 'development_stage', 'disease', 'donor_id', 'fresh_or_frozen', 'log10_total_counts', 'lung_condition', 'mixed_ancestry', 'original_ann_level_1', 'original_ann_level_2', 'original_ann_level_3', 'original_ann_level_4', 'original_ann_level_5', 'original_ann_nonharmonized', 'reannotation_type', 'sample', 'scanvi_label', 'sequencing_platform', 'sex', 'smoking_status', 'study', 'subject_type', 'suspension_type', 'tissue', 'tissue_coarse_unharmonized', 'tissue_detailed_unharmonized', 'tissue_dis

In [5]:
tmp = sc.AnnData(adata.obsm['X_scanvi_emb'])
tmp.obs = adata.obs
adata = tmp
del tmp
adata

AnnData object with n_obs × n_vars = 2282447 × 30
    obs: "3'_or_5'", 'BMI', 'age_or_mean_of_age_range', 'age_range', 'anatomical_region_ccf_score', 'ancestry', 'ann_coarse_for_GWAS_and_modeling', 'ann_finest_level', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'assay', 'cause_of_death', 'cell_type', 'core_or_extension', 'dataset', 'development_stage', 'disease', 'donor_id', 'fresh_or_frozen', 'log10_total_counts', 'lung_condition', 'mixed_ancestry', 'original_ann_level_1', 'original_ann_level_2', 'original_ann_level_3', 'original_ann_level_4', 'original_ann_level_5', 'original_ann_nonharmonized', 'reannotation_type', 'sample', 'scanvi_label', 'sequencing_platform', 'sex', 'smoking_status', 'study', 'subject_type', 'suspension_type', 'tissue', 'tissue_coarse_unharmonized', 'tissue_detailed_unharmonized', 'tissue_dissociation_protocol', 'tissue_level_2', 'tissue_level_3', 'tissue_sampling_method', 'total_counts', 'transf_ann_level_1_label', 'transf_ann_lev

## get rid of celltypes

In [6]:
adata = adata[~adata.obs['transf_ann_level_3_label'].isin(['2_Hematopoietic stem cells', '2_Mesothelium', '2_Smooth muscle', 'Rare'])].copy()
adata

AnnData object with n_obs × n_vars = 2250578 × 30
    obs: "3'_or_5'", 'BMI', 'age_or_mean_of_age_range', 'age_range', 'anatomical_region_ccf_score', 'ancestry', 'ann_coarse_for_GWAS_and_modeling', 'ann_finest_level', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'assay', 'cause_of_death', 'cell_type', 'core_or_extension', 'dataset', 'development_stage', 'disease', 'donor_id', 'fresh_or_frozen', 'log10_total_counts', 'lung_condition', 'mixed_ancestry', 'original_ann_level_1', 'original_ann_level_2', 'original_ann_level_3', 'original_ann_level_4', 'original_ann_level_5', 'original_ann_nonharmonized', 'reannotation_type', 'sample', 'scanvi_label', 'sequencing_platform', 'sex', 'smoking_status', 'study', 'subject_type', 'suspension_type', 'tissue', 'tissue_coarse_unharmonized', 'tissue_detailed_unharmonized', 'tissue_dissociation_protocol', 'tissue_level_2', 'tissue_level_3', 'tissue_sampling_method', 'total_counts', 'transf_ann_level_1_label', 'transf_ann_lev

## clean conditions

In [7]:
condition_simplifier = {
        "Healthy (tumor adjacent)": "Healthy",
        "Large Cell Carcinoma": "Carcinoma",
        "Lung adenocarcinoma": "Carcinoma",
        "Pleiomorphic Carcinoma": "Carcinoma",
        "Squamous Cell Carcinoma": "Carcinoma",
        "HP": "ILD",
        "Myositis-associated ILD": "ILD",
        "NSIP": "ILD",
        "Systemic sclerosis-associated ILD" : "ILD",
    }

In [8]:
adata.obs['lung_condition_coarse'] = adata.obs['lung_condition'].replace(condition_simplifier)
adata.obs['lung_condition_coarse'].cat.categories

Index(['CF', 'COPD', 'COVID-19', 'COVID-19 moderate', 'COVID-19 severe',
       'Chronic rhinitis', 'End-stage lung fibrosis, unknown etiology',
       'Healthy', 'ILD', 'IPF', 'Carcinoma', 'Lymphangioleiomyomatosis',
       'Pneumonia non-COVID', 'Sarcoidosis',
       'anti-CD3 anti-CD28 antibody stimulated'],
      dtype='object')

In [9]:
adata.obs[['lung_condition_coarse', 'sample']].drop_duplicates()['lung_condition_coarse'].value_counts()

Healthy                                      429
COVID-19                                      73
IPF                                           67
Carcinoma                                     44
COPD                                          24
CF                                            22
ILD                                           22
Chronic rhinitis                              18
Pneumonia non-COVID                           13
COVID-19 severe                                6
End-stage lung fibrosis, unknown etiology      4
COVID-19 moderate                              3
Lymphangioleiomyomatosis                       2
Sarcoidosis                                    2
anti-CD3 anti-CD28 antibody stimulated         2
Name: lung_condition_coarse, dtype: int64

In [10]:
conditions_to_exclude = [
    "End-stage lung fibrosis, unknown etiology",
    "Lymphangioleiomyomatosis",
    "Sarcoidosis",
    "anti-CD3 anti-CD28 antibody stimulated",
]

In [11]:
adata = adata[~adata.obs['lung_condition_coarse'].isin(conditions_to_exclude)].copy()
adata

AnnData object with n_obs × n_vars = 2212137 × 30
    obs: "3'_or_5'", 'BMI', 'age_or_mean_of_age_range', 'age_range', 'anatomical_region_ccf_score', 'ancestry', 'ann_coarse_for_GWAS_and_modeling', 'ann_finest_level', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'assay', 'cause_of_death', 'cell_type', 'core_or_extension', 'dataset', 'development_stage', 'disease', 'donor_id', 'fresh_or_frozen', 'log10_total_counts', 'lung_condition', 'mixed_ancestry', 'original_ann_level_1', 'original_ann_level_2', 'original_ann_level_3', 'original_ann_level_4', 'original_ann_level_5', 'original_ann_nonharmonized', 'reannotation_type', 'sample', 'scanvi_label', 'sequencing_platform', 'sex', 'smoking_status', 'study', 'subject_type', 'suspension_type', 'tissue', 'tissue_coarse_unharmonized', 'tissue_detailed_unharmonized', 'tissue_dissociation_protocol', 'tissue_level_2', 'tissue_level_3', 'tissue_sampling_method', 'total_counts', 'transf_ann_level_1_label', 'transf_ann_lev

## add splits

## separate query

In [12]:
adata_query = adata[adata.obs['dataset'] == 'Zhang_2021'].copy()
adata = adata[adata.obs['dataset'] != 'Zhang_2021'].copy()

## add same splits

In [13]:
adata_pearson = sc.read('/lustre/groups/ml01/projects/2022_multigrate_anastasia.litinetskaya/snakemake/hyperparam/data/pp/hlca_hvg_pearson.h5ad')
adata_pearson

AnnData object with n_obs × n_vars = 2149684 × 2000
    obs: "3'_or_5'", 'BMI', 'age_or_mean_of_age_range', 'age_range', 'anatomical_region_ccf_score', 'ancestry', 'ann_coarse_for_GWAS_and_modeling', 'ann_finest_level', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'assay', 'cause_of_death', 'cell_type', 'core_or_extension', 'dataset', 'development_stage', 'disease', 'donor_id', 'fresh_or_frozen', 'log10_total_counts', 'lung_condition', 'mixed_ancestry', 'original_ann_level_1', 'original_ann_level_2', 'original_ann_level_3', 'original_ann_level_4', 'original_ann_level_5', 'original_ann_nonharmonized', 'reannotation_type', 'sample', 'scanvi_label', 'sequencing_platform', 'sex', 'smoking_status', 'study', 'subject_type', 'suspension_type', 'tissue', 'tissue_coarse_unharmonized', 'tissue_detailed_unharmonized', 'tissue_dissociation_protocol', 'tissue_level_2', 'tissue_level_3', 'tissue_sampling_method', 'total_counts', 'transf_ann_level_1_label', 'transf_ann_l

In [14]:
adata.obs = adata.obs.join(adata_pearson.obs[[f'split{i}' for i in range(5)]])

In [15]:
adata.write('/lustre/groups/ml01/projects/2022_multigrate_anastasia.litinetskaya/snakemake/hyperparam/data/pp/hlca_embed.h5ad')
adata_query.write('/lustre/groups/ml01/projects/2022_multigrate_anastasia.litinetskaya/snakemake/hyperparam/data/pp/hlca_embed_query.h5ad')